## Llama 3.3 70B Instruct - SAE Inference

The following are multiple examples of getting activations and searching Llama 3.3 70B on Neuronpedia using [Goodfire's Layer 50 SAE](https://huggingface.co/Goodfire/Llama-3.3-70B-Instruct-SAE-l50/tree/main).

In the following examples, we focus on using feature 7336 from the layer 50 SAE.

*Dashboards are not yet uploaded for this SAE, so the links to Neuronpedia dashboards will not work.*

### Limitations
- Non-batch requests: Limit of 1024 tokens per prompt
- Batch requests: Limit of 256 tokens per prompt, with a max of 4 prompts per batch request
- Steering is not yet enabled - we expect this to be available soon - if you need this urgently, contact us at team@neuronpedia.org.

In [1]:
# ===== Constants =====

# We specify the model and SAE source ID
model_id = "llama3.3-70b-it"
source = "50-resid-post-gf"

# We'll use a "Rice" feature as an example
index = 7336

# Example single prompt
single_prompt = "I like to eat rice."

# Example multiple (batch) prompts
batch_prompts = [
    "I like to eat rice.",
    "I like to eat pizza.",
    "I like to eat sushi.",
    "I like to pet dogs.",
]

# ===== Imports and Config =====

%pip install neuronpedia

from dotenv import load_dotenv
from neuronpedia.requests.activation_request import ActivationRequest
from neuronpedia.requests.source_activation_request import SourceActivationRequest
from neuronpedia.requests.topk_by_token_request import TopKByTokenRequest

# Load NEURONPEDIA_API_KEY from .env file
load_dotenv()

### [less safe] Set Neuronpedia API key manually (get your key from neuronpedia.org/account)
# import os
# os.environ["NEURONPEDIA_API_KEY"] = "YOUR_KEY_HERE"


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


True

In [2]:
# Example 1: Activations for Single Feature, Single Prompt
# Maximum of 1024 tokens per prompt

activation = ActivationRequest().compute_activation_for_text(model_id, source, index, single_prompt)

print(activation)

Sending POST request to https://neuronpedia.org/api/activation/new
Got a successful response.
Activation for modelId: llama3.3-70b-it, source: 50-resid-post-gf, index: 7336
<|begin_of_text|>	0
I	0
 like	0
 to	0
 eat	0
 rice	15
.	0.466796875


In [3]:
# Example 2: Activations for Single Feature, Multiple Prompts (Batch)
# Maximum of 256 tokens per prompt, up to 4 prompts per request

activations = ActivationRequest().compute_activation_for_texts(model_id, source, index, batch_prompts)

for activation in activations:
    print(activation)


Sending POST request to https://neuronpedia.org/api/activation/new
Got a successful response.
Activation for modelId: llama3.3-70b-it, source: 50-resid-post-gf, index: 7336
<|begin_of_text|>	0
I	0
 like	0
 to	0
 eat	0
 rice	14.5625
.	0.02490234375
Activation for modelId: llama3.3-70b-it, source: 50-resid-post-gf, index: 7336
<|begin_of_text|>	0
I	0
 like	0
 to	0
 eat	0
 pizza	0
.	0
Activation for modelId: llama3.3-70b-it, source: 50-resid-post-gf, index: 7336
<|begin_of_text|>	0
I	0
 like	0
 to	0
 eat	0
 sushi	0
.	0
Activation for modelId: llama3.3-70b-it, source: 50-resid-post-gf, index: 7336
<|begin_of_text|>	0
I	0
 like	0
 to	0
 pet	0
 dogs	0
.	0


In [4]:
# Example 3: Activations for All Features in SAE, Single Prompt
# Maximum of 1024 tokens per prompt

# We sort the results by highest activation at "rice", the token at index 5.
# We can also exclude that argument if we want to see the top features for all tokens.
activations = SourceActivationRequest().compute_top_features_for_text(
    model_id,
    source,
    single_prompt,
    ignore_bos=True,
    sort_results_by_activations_at_token_indexes=[5],
    # see method for more options
)
for activation in activations:
    print("")
    print(activation)
    print(f"https://neuronpedia.org/{activation.modelId}/{activation.source}/{activation.index}")
    print("-" * 50)

Sending POST request to https://neuronpedia.org/api/search-all/
Got a successful response.

Activation for modelId: llama3.3-70b-it, source: 50-resid-post-gf, index: 7336
<|begin_of_text|>	0
I	0
 like	0
 to	0
 eat	0
 rice	14.5
.	0.01806640625
https://neuronpedia.org/llama3.3-70b-it/50-resid-post-gf/7336
--------------------------------------------------

Activation for modelId: llama3.3-70b-it, source: 50-resid-post-gf, index: 52054
<|begin_of_text|>	0
I	0
 like	0
 to	0
 eat	0.4453125
 rice	2.203125
.	0.56640625
https://neuronpedia.org/llama3.3-70b-it/50-resid-post-gf/52054
--------------------------------------------------

Activation for modelId: llama3.3-70b-it, source: 50-resid-post-gf, index: 56767
<|begin_of_text|>	0
I	0
 like	0.1220703125
 to	0
 eat	4.09375
 rice	2.0625
.	1.1015625
https://neuronpedia.org/llama3.3-70b-it/50-resid-post-gf/56767
--------------------------------------------------

Activation for modelId: llama3.3-70b-it, source: 50-resid-post-gf, index: 3581
<|begi

In [5]:
# Example 4: Activations for All Features in SAE, Multiple Prompts (batch)
# Maximum of 256 tokens per prompt, up to 4 prompts per request

activations_batch = SourceActivationRequest().compute_top_features_for_texts(
    model_id,
    source,
    batch_prompts,
    ignore_bos=True,
    # see method for more options
)

for i, activations in enumerate(activations_batch):
    print("")
    print(f"Prompt: {batch_prompts[i]}")
    print("=" * 50) 
    for activation in activations:
        print("")
        print(activation)
        print(f"https://neuronpedia.org/{activation.modelId}/{activation.source}/{activation.index}")
        print("-" * 50)

Sending POST request to https://neuronpedia.org/api/search-all/
Got a successful response.

Prompt: I like to eat rice.

Activation for modelId: llama3.3-70b-it, source: 50-resid-post-gf, index: 7336
<|begin_of_text|>	0
I	0
 like	0
 to	0
 eat	0
 rice	14.5625
.	0.02490234375
https://neuronpedia.org/llama3.3-70b-it/50-resid-post-gf/7336
--------------------------------------------------

Activation for modelId: llama3.3-70b-it, source: 50-resid-post-gf, index: 57413
<|begin_of_text|>	0
I	0
 like	0
 to	0
 eat	12.6875
 rice	0
.	0
https://neuronpedia.org/llama3.3-70b-it/50-resid-post-gf/57413
--------------------------------------------------

Activation for modelId: llama3.3-70b-it, source: 50-resid-post-gf, index: 25595
<|begin_of_text|>	0
I	0
 like	0.46875
 to	11.875
 eat	2.34375
 rice	0
.	0
https://neuronpedia.org/llama3.3-70b-it/50-resid-post-gf/25595
--------------------------------------------------

Activation for modelId: llama3.3-70b-it, source: 50-resid-post-gf, index: 10920
<|be

In [6]:
# Example 5: TopK by Token for All Features in SAE, Single Prompt
# Maximum of 1024 tokens per prompt

topk_by_token = TopKByTokenRequest().compute_top_features_by_token_for_text(
    model_id,
    source,
    single_prompt,
    num_results=5,
    ignore_bos=True,
)

for result in topk_by_token:
    print(f"\nToken: '{result.token}' at position {result.position}")
    print(f"Top {len(result.topFeatures)} features:")
    for i, feature_activation in enumerate(result.topFeatures, 1):
        print(f"  {i}. Feature {feature_activation.feature.index}: {feature_activation.activationValue:.4f}")
        print(f"     https://neuronpedia.org/{feature_activation.feature.model}/{feature_activation.feature.source}/{feature_activation.feature.index}")

Sending POST request to https://neuronpedia.org/api/search-topk-by-token/
Got a successful response.

Token: 'I' at position 0
Top 5 features:
  1. Feature 4735: 0.5273
     https://neuronpedia.org/llama3.3-70b-it/50-resid-post-gf/4735
  2. Feature 9189: 0.3145
     https://neuronpedia.org/llama3.3-70b-it/50-resid-post-gf/9189
  3. Feature 17658: 0.2637
     https://neuronpedia.org/llama3.3-70b-it/50-resid-post-gf/17658
  4. Feature 1892: 0.1904
     https://neuronpedia.org/llama3.3-70b-it/50-resid-post-gf/1892
  5. Feature 57139: 0.0693
     https://neuronpedia.org/llama3.3-70b-it/50-resid-post-gf/57139

Token: ' like' at position 1
Top 5 features:
  1. Feature 10920: 9.6875
     https://neuronpedia.org/llama3.3-70b-it/50-resid-post-gf/10920
  2. Feature 17658: 3.3594
     https://neuronpedia.org/llama3.3-70b-it/50-resid-post-gf/17658
  3. Feature 3934: 2.4219
     https://neuronpedia.org/llama3.3-70b-it/50-resid-post-gf/3934
  4. Feature 16043: 1.8047
     https://neuronpedia.org/lla

In [7]:
# Example 6: TopK by Token for All Features in SAE, Multiple Prompts (batch)    
# Maximum of 256 tokens per prompt, up to 4 prompts per request

topk_by_token_batch = TopKByTokenRequest().compute_top_features_by_token_for_texts(
    model_id,
    source,
    batch_prompts,
    num_results=5,
    ignore_bos=True,
)

for batch_idx, topk_by_token in enumerate(topk_by_token_batch):
    print(f"\nPrompt {batch_idx + 1}: {batch_prompts[batch_idx]}")
    print("=" * 50)
    for result in topk_by_token:
        print(f"\nToken: '{result.token}' at position {result.position}")
        print(f"Top {len(result.topFeatures)} features:")
        for i, feature_activation in enumerate(result.topFeatures, 1):
            print(f"  {i}. Feature {feature_activation.feature.index}: {feature_activation.activationValue:.4f}")
            print(f"     https://neuronpedia.org/{feature_activation.feature.model}/{feature_activation.feature.source}/{feature_activation.feature.index}")

Sending POST request to https://neuronpedia.org/api/search-topk-by-token/
Got a successful response.

Prompt 1: I like to eat rice.

Token: 'I' at position 0
Top 5 features:
  1. Feature 4735: 0.5234
     https://neuronpedia.org/llama3.3-70b-it/50-resid-post-gf/4735
  2. Feature 9189: 0.3047
     https://neuronpedia.org/llama3.3-70b-it/50-resid-post-gf/9189
  3. Feature 17658: 0.2793
     https://neuronpedia.org/llama3.3-70b-it/50-resid-post-gf/17658
  4. Feature 1892: 0.1787
     https://neuronpedia.org/llama3.3-70b-it/50-resid-post-gf/1892
  5. Feature 57139: 0.0596
     https://neuronpedia.org/llama3.3-70b-it/50-resid-post-gf/57139

Token: ' like' at position 1
Top 5 features:
  1. Feature 10920: 9.6875
     https://neuronpedia.org/llama3.3-70b-it/50-resid-post-gf/10920
  2. Feature 17658: 3.3594
     https://neuronpedia.org/llama3.3-70b-it/50-resid-post-gf/17658
  3. Feature 3934: 2.4219
     https://neuronpedia.org/llama3.3-70b-it/50-resid-post-gf/3934
  4. Feature 16043: 1.8047
 